In [ ]:
import pandas

In [ ]:
# default_exp proc

In [ ]:
#hide
# "Hide" blir ikke med i hverken eksportert modul eller dokumentasjonen
from nbdev.showdoc import *

# SAS-ish functions

> Mimic some of the missed functionality from SAS

### FREQ
https://medium.com/eduonline24/can-we-get-sas-proc-freq-with-python-c6af752f387a

In [ ]:
# FREQ
# "Missing"?
# Verbose

In [ ]:
#export
from IPython.display import display
import pandas as pd

def freq(df, cols = [], order = 'data', nopercent = False, nocum = False, freq_limit = 0):
    
    """
    This function tries to mimic some of the functionality of "PROC FREQ" from SAS.\n
    When using crosstab, by using an asterix in a string 'col1*col2', only frequencies are included.\n
    You may use numbers for columns, even negative ones, with an asterix. For example: cols = 'col1*-1', for the column "col1" crosstabbed with the last column.
    
    
    Parameters
    ----------
    first "df" : dataframe\n
        Send in the pandas dataframe you want to analyze.
    second "cols" : string, int or list of ints and or strings\n
        Strings may also contain an asterisk '*' between two column-names / ints. To create crosstabs of two columns.
        A list of several strings / ints, will generate multiple frequency-tables from the same dataframe and 
    third "order" : string\n
        Only possible alternative to the default 'data' is currently 'freq', actually, if passed anything other than 'data', it will sort by frequency.
    fourth "nopercent" : bool\n
        Will remove the columns 'Percent' and 'Cumulative Percent' if True
    fifth "nocum" : bool\n
        Will remove the columns 'Cumulative Frequency' and 'Cumulative Percent' if True
    sixth "freq_limit" : int \n
        Lower limit for frequency-rows in non-crosstabbed tables. It will remove rows with frequency values below this.

    Returns
    -------
    None\n
        Only displays the frequency tables, does not return a dataframe.

    Raises
    ------
    ValueError\n
        If no cols-parameter is specified.
    """  
    
    
    # Make a copy of the dataframe, not to mess it up
    dft = df.copy()
    
    # We need to know which tables to look at
    # 0 alone is a valid input, but not empty strings or lists
    if not cols and cols != 0:
        raise ValueError('Please pass a "cols" parameter witht he columns youd like to take a look at.')
    
    # Convert cols to list, if its a string
    cols = [cols] if isinstance(cols, str) or isinstance(cols, int) else cols
    
    # Loop over all the cols you want
    for col in cols:
        #print(col)
        # If int try to get that numbered column
        if isinstance(col, int):
            col = dft.columns[col]
        # If string, try to pick that column
              
        # What if there is a star in the string?   
        if col.find('*') !=-1 :
            
            # Clean up ints / strings on their way in with asteriks
            col0 = col.split('*')[0]
            try:
                col0 = int(col0)
                if col0 < 0: col0 = dft.shape[1] - col0 - 2 
                col0 = dft.columns[col0]
            except:
                ...
            
            col1 = col.split('*')[1]
            try:
                col1 = int(col1)
                if col1 < 0: col1 = dft.shape[1] - col1 - 2
                col1 = dft.columns[col1]
            except:
                ...
            
            # Print
            print(col0, '*', col1)
            
            # Crosstab-function
            datab = pd.crosstab(dft[col0], dft[col1], margins=True, margins_name="Total")
            
            display(datab)
            # We will exit here if crosstab, since it becomes too complicated otherwise?
            
        else:
            dfts = dft[col]

            datax = dfts.value_counts()
            if order == 'data': 
                datax = datax.sort_index()
            datay = pd.DataFrame({
                    col: datax.index,
                    'Frequency': datax.values,
                    'Percent': ((datax.values/datax.values.sum())*100).round(2),
                    'Cumulative Frequency': datax.values.cumsum(),
                    'Cumulative Percent': ((datax.values.cumsum()/datax.values.sum())*100).round(2)
                    })
            
            if nopercent:
                datay = datay.drop('Percent', axis = 1)
            if nocum:
                datay = datay.drop('Cumulative Frequency', axis = 1)
            if nocum or nopercent:
                datay = datay.drop('Cumulative Percent', axis = 1)
            
            if freq_limit and isinstance(freq_limit, int):
                datay = datay[datay['Frequency'] >= freq_limit]
            
            display(datay.style.hide_index())
    
    return None

In [ ]:
#dapla
# Example usage at SSB
import dapla as dp
df = dp.read_pandas('/felles/mock_sysselsatte/companies_2020_10000')

In [ ]:
for col in sorted(list(df.columns)): print(col)

employee_points
nace
region
region_code
work_id


In [ ]:
#dapla
# Freq of last column
freq(df, -1)

employee_points,Frequency,Percent,Cumulative Frequency,Cumulative Percent
1,31,7.730000,31,7.730000
2,12,2.990000,43,10.720000
3,46,11.470000,89,22.190000
4,43,10.720000,132,32.920000
5,25,6.230000,157,39.150000
6,21,5.240000,178,44.390000
7,23,5.740000,201,50.120000
8,30,7.480000,231,57.610000
9,11,2.740000,242,60.350000
10,15,3.740000,257,64.090000


In [ ]:
#dapla
# Cross table of 'employee_points' and 'the first column'
freq(df, cols = 'employee_points*0')

In [ ]:
#dapla
# Minimal return
freq(df, cols = ['employee_points'], 
     # No Percent column
     nopercent = True, 
     # No Cumulative Freq
     nocum = True, 
     # Either of the ones above will remove the "cumulative percent" column
     # Lower limit of frequencies to show
     freq_limit = 30)

employee_points,Frequency
1,31
3,46
4,43
8,30


In [ ]:
# CONTENTS
# List opp nyttig info



In [ ]:
from pandas.api.types import CategoricalDtype

def category_range_convert(series, labels):
    
    # Check if series is a categorical already...
        # What should we do then?
    # If series is not already categorical:
        # Find max integer value in labels / series
        # Make sure series only contains ints
        # series.astype(CategoricalDtype(categories = list(range(max_int_value + 1)), ordered = True))
    
    
    
    # Return series

def labels_apply(series, labels, flip = False):
    
    
    ### TYPE CHECKING ####
    # Check that series is a pandas series
    if not isinstance(series, pd.Series):
        raise ValueError('First parameter is not a pandas series, please pass in a series.')
    # Check that labels is a dict, or a series itself
    if (not isinstance(labels, dict)) or (not isinstance(labels, pd.Series)):
        raise ValueError('Labels-parameter, second, is not a dict or a pandas series.')
    # If it is a series, check that it is a categorical
    if isinstance(labels, pd.Series):
        if labels.dtype.name != "category":
            raise ValueError('The pandas Series passed in as "lables" is not itself a categorical, so there is no category-settings to copy.')
    # Check that flip is a bool
    if not isinstance(flip, bool):
        raise ValueError('Flip, third parameter, is not a bool (True/False)')
    
    # Check which way the column to be converted is flipped (contains strings or not)
    # If it does not match flip-parameter, reccommend changing the parameter, or make aware that function may have been re-run on same column
    
    # If labels is a dict
        # Look for dict in dict with same name as column.upper
            # If not found check if we can use the flat dict (values should be pure int/strings, not dicts etc.)
        # One side, keys/values, needs to be pure integers
        # Check which way labels is flipped
            # Consider flipping the dict to match what the series contains
        # If column is to be converted to categorical
            # category_range_convert(series)
        # If column is Categorical
            # .cat.rename_categories()
        # If column is not categorical
            # replace values with the other side of the dict
    # If labels is a categorical column itself
        # If all values in column are represented in the original Categorical
            # Identify the settings of the labels-cat
            # Easy way to copy these?
        # If the Categoricals dont match up, we might have to do a complicated Union somehow?
        
        
    # Return the modified series

In [ ]:
# GLM regresjon - dummies
# 